In [1]:
# 필요한 모듈 임포트
!pip install snscrape
import snscrape.modules.twitter as sntwitter
import pandas as pd

!pip install konlpy
from konlpy.tag import Okt
okt = Okt()

import re

In [2]:
# Query by text search
# Setting variables to be used below
maxTweets = 5000

# Creating list to append tweet data to
tweets_list = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('느린마을 막걸리 since:2019-01-01 until:2021-12-31').get_items()):
    if i>maxTweets:
        break
    tweets_list.append([tweet.date, tweet.id, tweet.content, tweet.user.username])

# Creating a dataframe from the tweets list above
tweets_df = pd.DataFrame(tweets_list, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])

# 텍스트만 가져오기
tweets_text = [tweet[2] for tweet in tweets_list]

In [3]:
# 정규표현식으로 정제하기
# url과 태그 없애기

reg1 = re.compile(r'https?://[a-zA-Z0-9_/:%#\$&\?\(\)~\.=+-]*') # url
reg2 = re.compile(r'(@)[a-zA-Z0-9_]*:*') # 계정 태그(@아이디)

def parge_tweet(tweet):
    t = tweet
    t = reg1.sub('', t) # url -> 삭제
    t = reg2.sub('', t) # 계정 태그(@아이디) -> 삭제
    return t

tweets = [parge_tweet(tweet) for tweet in tweets_text]

In [4]:
# 한국어 정규화 후 형태소 분석
okt_tweets = [okt.morphs(okt.normalize(tweet)) for tweet in tweets]

In [5]:
# 불용어 사전 불러오기
stop_words = []

with open("stopWords.txt", "r",encoding='UTF8') as f:
    data = f.read()

stop_words = data.split('\n')

with open("my_stopWords.txt", "r",encoding='UTF8') as f:
    data2 = f.read()

stop_words.extend(data2.split('\n'))

stop_words

['아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다',
 '의거하여',
 '근거하여',
 '입각하여',
 '기준으로',
 '예하면',
 '예를 들면',
 '예를 들자면',
 '저',
 '소인',
 '소생',
 '저희',
 '지말고',
 '하지마',
 '하지마라',
 '다른',
 '물론',
 '또한',
 '그리고',
 '비길수 없다',
 '해서는 안된다',
 '뿐만 아니라',
 '만이 아니다',
 '만은 아니다',
 '막론하고',
 '관계없이',
 '그치지 않다',
 '그러나',
 '그런데',
 '하지만',
 '든간에',
 '논하지 않다',
 '따지지 않다',
 '설사',
 '비록',
 '더라도',
 '아니면',
 '만 못하다',
 '하는 편이 낫다',
 '불문하고',
 '향하여',
 '향해서',
 '향하다',
 '쪽으로',
 '틈타',
 '이용하여',
 '타다',
 '오르다',
 '제외하고',
 '이 외에',
 '이 밖에',
 '하여야',
 '비로소',
 '한다면 몰라도',
 '외에도',
 '이곳',
 '여기',
 '부터',
 '기점으로',
 '따라서',
 '할 생각이다',
 '하려고하다',
 '이리하여',
 '그리하여',
 '그렇게 함으로써',
 '하지만',
 '일때',
 '할때',
 '앞에서',
 '중에서',
 '보는데서',
 '으로써',
 '로써',
 '까지',
 '해야한다',
 '일것이다',
 '반드시',
 '할줄알다',
 '할수있다',
 '할수있어',
 '임에 틀림없다',
 '한다면',
 '등',
 '등등',
 '제',
 '겨우',
 '단지',
 '다만',
 '할뿐',
 '딩동',
 '댕그',
 '대해서',
 '대하여',
 '대하면',
 '훨씬',
 '얼마나',
 '얼마만큼',
 '얼마큼',
 '남짓',
 '여',
 '얼마간',
 '약간',
 '다소',
 '좀',
 '조

In [6]:
# 기본 불용어 사전에 불용어 추가하기
user_stop_words = ['amp','旅がえり','gt','느린','마을','막걸리']
stop_words.extend(user_stop_words)

In [7]:
# 토큰 만들기
# stop_words 제거, 글자수 2개 이상, 한글 및 영어만
word_tokens = [word for tweet in okt_tweets for word in tweet \
               if word not in stop_words and len(word) > 1 and word.isalpha() \
               and 'ㅠ' not in word and 'ㅎ' not in word and 'ㅋ' not in word and 'ㅜ' not in word]

In [8]:
#특수문자 정제하기
import string
punct = list(string.punctuation)

word_tokens = [word for word in word_tokens if word not in punct]

In [9]:
 # -*- coding: utf8 -*- 
    
from collections import Counter

vocab = Counter()

for word in word_tokens:
    vocab[word] += 1

sorted(vocab.items(),key=lambda x: x[1], reverse=True)

[('추천', 108),
 ('마셔', 100),
 ('안주', 91),
 ('배상면', 85),
 ('지평', 82),
 ('마트', 73),
 ('존맛', 69),
 ('먹고', 68),
 ('여름', 64),
 ('마시고', 62),
 ('소주', 60),
 ('달달', 59),
 ('최고', 59),
 ('맥주', 59),
 ('맛있어요', 59),
 ('장수', 55),
 ('탄산', 54),
 ('최애', 53),
 ('맛있다', 51),
 ('가격', 48),
 ('순도', 46),
 ('편의점', 46),
 ('아스파탐', 44),
 ('친구', 43),
 ('별로', 43),
 ('많이', 40),
 ('가을', 40),
 ('파전', 40),
 ('정말', 39),
 ('겨울', 38),
 ('맛있어', 37),
 ('알밤', 36),
 ('느낌', 36),
 ('단맛', 35),
 ('맛있음', 35),
 ('김치전', 34),
 ('국순당', 33),
 ('사랑', 31),
 ('딸기', 30),
 ('와인', 30),
 ('처음', 28),
 ('좋아하는', 28),
 ('저녁', 27),
 ('취향', 27),
 ('해창', 26),
 ('전통주', 26),
 ('오랜', 25),
 ('이마트', 25),
 ('매장', 25),
 ('강남', 24),
 ('입맛', 24),
 ('숙취', 24),
 ('종류', 24),
 ('맛있는데', 23),
 ('술집', 23),
 ('좋아하는데', 22),
 ('우유', 22),
 ('메모', 22),
 ('동네', 22),
 ('엄마', 22),
 ('좋은', 22),
 ('한국', 22),
 ('유통', 21),
 ('코로나', 21),
 ('엄청', 21),
 ('맛있고', 21),
 ('서울', 21),
 ('감자전', 21),
 ('한잔', 21),
 ('부드럽고', 21),
 ('판매', 20),
 ('배송', 20),
 ('먹고싶다', 20),
 ('완전', 20),
 ('가게', 